In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils
import time

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [24]:
dat = pd.read_csv('../songdata.csv', usecols=['text'], nrows=10).drop_duplicates()

In [13]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [14]:
dat.loc[:, 'text'] += '(end)'

In [15]:
unique_chars = dat.loc[:, 'text'].apply(lambda x: list(set(x)))

In [16]:
unique_chars.head()

0    [a, z, A, s, l, p, S, f, b, n, o, ), y, i, (, ...
1    [a, z, A, s, p, l, f, b, n, o, ), O, y, i, (, ...
2    [a, A, l, p, s, S, B, f, b, n, o, ), y, i, (, ...
3    [a, A, s, p, l, S, B, f, b, n, o, O, ), y, i, ...
4    [a, A, s, p, l, S, B, f, b, n, o, O, ), y, i, ...
Name: text, dtype: object

In [17]:
unique_chars = unique_chars.sum()

In [18]:
unique_chars = sorted(np.unique(unique_chars))

In [19]:
len(unique_chars)

76

In [3]:
def scanning(text, length):
    encoder_input = []
    decoder_input = []
    text_length = len(text)
    for i in range(0, text_length-2*length+1):
        encoder_input.append(list(text[i:i+length]))
        decoder_input.append(list(text[i+length:i+2*length]))
    return [encoder_input, decoder_input]

In [5]:
word2ind, ind2word = utils.load_index_word_map()

In [6]:
%%time
dat_iter = pd.read_csv('../songdata.csv', usecols=['text'], chunksize=1000)
for_encoder = []
for_decoder = []
for ind, dat in enumerate(dat_iter):
    time_start = time.time()
    dat.loc[:, 'text'] += '(end)'
    sum_ser = dat.text.apply(lambda text: pd.Series(scanning(text, length=100), index=['for_encoder', 'for_decoder'])).sum()
    for_encoder_se = pd.Series(sum_ser.for_encoder)
    for_decoder_se = pd.Series(sum_ser.for_decoder)
    encoder_input_tokenised = utils.tokenise_cleaned_data(for_encoder_se, word2ind)
    decoder_input_tokenised = utils.tokenise_cleaned_data(for_decoder_se, word2ind)
    decoder_target_tokenised = decoder_input_tokenised.copy()
    decoder_input_tokenised = decoder_input_tokenised.apply(lambda x: [word2ind[' ']]+x[:-1])
    encoder_input_tokenised = pd.DataFrame(encoder_input_tokenised.values.tolist())
    decoder_input_tokenised = pd.DataFrame(decoder_input_tokenised.values.tolist())
    decoder_target_tokenised = pd.DataFrame(decoder_target_tokenised.values.tolist())
    encoder_input_tokenised.to_csv('data/encoder_input_{:02d}.csv'.format(ind), index=False)
    decoder_input_tokenised.to_csv('data/decoder_input_{:02d}.csv'.format(ind), index=False)
    decoder_target_tokenised.to_csv('data/decoder_target_{:02d}.csv'.format(ind), index=False)
    time_stop = time.time()
    print(ind, time_stop - time_start)

0 183.60801649093628
1 176.77477645874023
2 183.21090602874756
3 160.8573935031891
4 173.21614909172058
5 186.05057621002197
6 170.08350205421448
7 167.41624093055725
8 206.2008502483368
9 182.70949602127075
10 201.82492876052856
11 202.77631855010986
12 185.14270853996277
13 178.04063200950623
14 189.43374252319336
15 197.8701069355011
16 195.66886830329895
17 188.52455258369446
18 177.9999771118164
19 185.10700249671936
20 187.63849353790283
21 195.20005917549133
22 219.81669449806213
23 168.57537508010864
24 191.20675826072693
25 177.61986708641052
26 187.3392686843872
27 185.94926524162292
28 179.21534776687622
29 172.3378233909607
30 207.19139504432678
31 178.06361031532288
32 202.2091088294983
33 173.70639085769653
34 198.87040090560913
35 159.23995161056519
36 224.4864203929901
37 223.93734288215637
38 191.61143684387207
39 236.27651834487915
40 201.93576955795288
41 266.252325296402
42 198.9865608215332
43 201.24600911140442
44 173.7312090396881
45 191.04172372817993
46 186.508

In [17]:
utils.save_unique_tokens(unique_chars)

In [18]:
word2ind, ind2word = utils.get_index_word_map(unique_chars)

In [19]:
utils.save_index_word_map(word2ind, ind2word)

In [20]:
encoder_input_tokenised = utils.tokenise_cleaned_data(for_encoder_se, word2ind)

In [21]:
encoder_input_tokenised.head()

0    [33, 64, 64, 60, 1, 50, 69, 1, 57, 54, 67, 1, ...
1    [64, 64, 60, 1, 50, 69, 1, 57, 54, 67, 1, 55, ...
2    [64, 60, 1, 50, 69, 1, 57, 54, 67, 1, 55, 50, ...
3    [60, 1, 50, 69, 1, 57, 54, 67, 1, 55, 50, 52, ...
4    [1, 50, 69, 1, 57, 54, 67, 1, 55, 50, 52, 54, ...
dtype: object

In [22]:
decoder_input_tokenised = utils.tokenise_cleaned_data(for_decoder_se, word2ind)

In [23]:
decoder_input_tokenised.head()

0    [69, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68,...
1    [1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, ...
2    [68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72,...
3    [57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57,...
4    [54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57, 54,...
dtype: object

In [24]:
decoder_target_tokenised = decoder_input_tokenised.copy()

In [25]:
decoder_target_tokenised.head()

0    [69, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68,...
1    [1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, ...
2    [68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72,...
3    [57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57,...
4    [54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57, 54,...
dtype: object

In [26]:
len(decoder_target_tokenised.loc[0])

100

In [27]:
decoder_input_tokenised = decoder_input_tokenised.apply(lambda x: [word2ind[' ']]+x[:-1])

In [28]:
decoder_input_tokenised.head()

0    [1, 69, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, ...
1    [1, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, ...
2    [1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, ...
3    [1, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, ...
4    [1, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57, ...
dtype: object

In [29]:
len(decoder_input_tokenised.iloc[0])

100

In [30]:
%%time
encoder_input_tokenised = pd.DataFrame(encoder_input_tokenised.values.tolist())
decoder_input_tokenised = pd.DataFrame(decoder_input_tokenised.values.tolist())
decoder_target_tokenised = pd.DataFrame(decoder_target_tokenised.values.tolist())

In [31]:
encoder_input_tokenised.shape

(1898365, 100)

In [32]:
decoder_input_tokenised.shape

(1898365, 100)

In [33]:
decoder_target_tokenised.shape

(1898365, 100)

In [34]:
encoder_input_tokenised.to_csv('encoder_input.csv', index=False)
decoder_input_tokenised.to_csv('decoder_input.csv', index=False)
decoder_target_tokenised.to_csv('decoder_target.csv', index=False)